In [46]:
from skimage import io, exposure, img_as_float
from glob import glob
import os
import numpy as np
from tqdm import tqdm
import imageio
import torch
from torch import nn
from torchinfo import summary
from skimage.util import view_as_blocks, view_as_windows
from sklearn.cluster import KMeans
from PIL import Image as im
import sys

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
np.set_printoptions(threshold=sys.maxsize)
WINDOW_SHAPE = (64, 64)

In [11]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.fc = nn.Identity()

model = model.to(device)

resnet_summary = summary(model, (16,3,384,384))
print(resnet_summary)

Using cache found in /home/snamburi/.cache/torch/hub/pytorch_vision_v0.10.0


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   --                        --
├─Conv2d: 1-1                            [16, 64, 192, 192]        9,408
├─BatchNorm2d: 1-2                       [16, 64, 192, 192]        128
├─ReLU: 1-3                              [16, 64, 192, 192]        --
├─MaxPool2d: 1-4                         [16, 64, 96, 96]          --
├─Sequential: 1-5                        [16, 256, 96, 96]         --
│    └─Bottleneck: 2-1                   [16, 256, 96, 96]         --
│    │    └─Conv2d: 3-1                  [16, 64, 96, 96]          4,096
│    │    └─BatchNorm2d: 3-2             [16, 64, 96, 96]          128
│    │    └─ReLU: 3-3                    [16, 64, 96, 96]          --
│    │    └─Conv2d: 3-4                  [16, 64, 96, 96]          36,864
│    │    └─BatchNorm2d: 3-5             [16, 64, 96, 96]          128
│    │    └─ReLU: 3-6                    [16, 64, 96, 96]          --
│ 

In [3]:
input_files = glob('raw_data/*')
output_path = 'processed_image'

In [43]:
im_name = 'processed_stack.tif'
save_name = os.path.join(output_path, im_name)
final_list  =[]
for input_file in tqdm(input_files):
    im = io.imread(input_file)
    print(im.shape)
    for i in range(im.shape[0]):
        max_val = np.max(im[i])
        min_val = np.min(im[i])
#         print(max_val, min_val)
        processed_img = exposure.rescale_intensity(im[i], in_range=(min_val, max_val), 
                                                   out_range=(0, 1))
        final_list.append(processed_img)
#         print(np.max(processed_img), np.min(processed_img))
#         print("--------------------------")
final_stack = np.asarray(final_list)
io.imsave(save_name, final_stack) 

  0%|                                                     | 0/1 [00:00<?, ?it/s]

(10, 2048, 2048)
[112 110 115 116 109 110 142 112 109 126]
[112 113 112 110 113 112 134 113 136 116]
[113 110 113 110 113 113 137 114 134 118]
[114 110 113 111 113 112 137 113 109 123]
[117 109 110 109 119 110 158 116 132 118]
[115 109 111 112 113 113 135 113 108 118]


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.08it/s]

[112 112 115 113 113 112 144 112 113 119]
[114 111 116 112 112 112 138 110 110 115]
[111 108 115 111 115 110 140 116 140 111]
[116 112 113 112 116 111 152 115 108 121]


In [5]:
# for input_file in tqdm(input_files):
#     im = io.imread(input_file)
#     img = im[4]
#     im_p = exposure.rescale_intensity(img_as_float(img), in_range=(min_val, max_val), out_range=(0, 1))
#     im_name = f"processed_stack4.png"
#     save_name = os.path.join(output_path, im_name)
#     io.imsave(save_name, im_p)

In [41]:
final_stack[0][0][0:10], final_stack[1][0][0:10]

(array([0.0120919 , 0.00967352, 0.01571947, 0.01692866, 0.00846433,
        0.00967352, 0.04836759, 0.0120919 , 0.00846433, 0.02902056]),
 array([0.01262916, 0.01377727, 0.01262916, 0.01033295, 0.01377727,
        0.01262916, 0.03788749, 0.01377727, 0.0401837 , 0.01722158]))

In [47]:
for i in range(1):
    # The input image.
    image = final_stack[4]
    patches = view_as_windows(image, WINDOW_SHAPE, step=(8, 8))
    feature_matrix = [[0 for _ in range(patches.shape[0])] for _ in range(patches.shape[1])]
    for row in range(patches.shape[0]):
        for col in range(patches.shape[1]):
            patch_tensor = torch.tensor(patches[row][col]).float()
            patch_tensor = patch_tensor.repeat(3,1,1)
            features = model(patch_tensor[None, ...].to(device))
            feature_matrix[row][col] = features.cpu().detach().numpy()

In [48]:
patches.shape

(249, 249, 64, 64)

In [15]:
feature_matrix[1][0]

array([[0.50679195, 0.33065122, 0.49684864, 0.32950777, 0.37421784,
        0.38602367, 0.5302409 , 0.43154693, 0.6766212 , 0.37623623,
        0.682007  , 0.6295311 , 0.4139775 , 0.38590625, 0.37710407,
        0.13015813, 0.1762693 , 0.2333796 , 0.43182015, 0.22681266,
        0.49171752, 0.45707285, 0.49277753, 0.42542547, 0.5185395 ,
        0.69897115, 0.47581622, 0.43730134, 0.66322964, 0.40245423,
        0.51143754, 0.13690142, 0.20362681, 0.31415325, 0.31194222,
        0.5477159 , 0.24460438, 0.2479581 , 0.319166  , 0.4745433 ,
        0.45460987, 0.23922233, 0.5924244 , 0.5226075 , 0.66243505,
        0.37708673, 0.26959774, 0.6014857 , 0.14320703, 0.5140646 ,
        0.35225087, 0.38537875, 0.5412837 , 0.3271964 , 0.58725893,
        0.14892593, 0.4562782 , 0.50267375, 0.5403886 , 0.33765024,
        0.35722712, 0.4692354 , 0.270267  , 0.33213246, 0.42934847,
        0.28607938, 0.4792844 , 0.47041264, 0.42837757, 0.36653638,
        0.23836786, 0.33871287, 0.40633476, 0.43

In [49]:
feature_matrix_reshaped = np.reshape(feature_matrix, (len(feature_matrix) * len(feature_matrix[0]), feature_matrix[0][0].shape[1]))

In [50]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(feature_matrix_reshaped)

In [51]:
k_means_label_reshaped = np.reshape(kmeans.labels_, (len(feature_matrix), len(feature_matrix[0])))

In [58]:
k_means_label_reshaped.shape, type(k_means_label_reshaped)

((249, 249), numpy.ndarray)

In [71]:
imageio.imwrite('kmeans_dl.png', k_means_label_reshaped)

Lossy conversion from int32 to uint8. Range [0, 3]. Convert image to uint8 prior to saving to suppress this warning.


In [72]:
img = im.open(r"kmeans_dl.png")
img = img.resize((2048,2048), im.BILINEAR)
imageio.imwrite('resized.png', img)

In [62]:
color_value = 255/4
out_image = np.empty(image.shape)
def coloring():
    for i in range(k_means_label_reshaped.shape[0]):
        for j in range(k_means_label_reshaped.shape[1]):
            out_image[64*i:64*i+64, 64*j:64*j+64] = k_means_label_reshaped[i][j] * color_value

coloring()

In [63]:
type(out_image)

numpy.ndarray

In [64]:
imageio.imwrite('kmeans_dl.png', out_image)

Lossy conversion from float64 to uint8. Range [0.0, 191.25]. Convert image to uint8 prior to saving to suppress this warning.
